# Felipe Castillo
# Housing Train Assingment
# Data Mining
# 04/30/2022

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler , StandardScaler
#Change path to week one folder for DSC-550
#Current directory in documents
os.chdir('C:\DataScience_DSC_550\Week7\HousingFolder')


# 1.Import the housing data as a data frame and ensure that the data is loaded properly.

In [2]:
#Loading Data
housing_df = pd.read_csv("train.csv")

In [3]:
housing_df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


# 2.Drop the "Id" column and any features that are missing more than 40% of their values.

In [4]:
#Dropping Id Columnn
housing_df = housing_df.drop('Id', axis=1)

In [5]:
housing_df.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [6]:

#Getting a percentage of missing values that each column has
def missing_percentage(df):
    print ((df.isnull().sum() * 100 / len(df)).sort_values(ascending=False))

    
missing_percentage(housing_df)

PoolQC         99.520548
MiscFeature    96.301370
Alley          93.767123
Fence          80.753425
FireplaceQu    47.260274
                 ...    
Heating         0.000000
HeatingQC       0.000000
MSZoning        0.000000
1stFlrSF        0.000000
SalePrice       0.000000
Length: 80, dtype: float64


In [7]:
#Droping columns that are missing 40% of their values
housing_df = housing_df[housing_df.columns[housing_df.isnull().mean() <.40]]

#Rechecking values after drop, to double check function
housing_df.isnull().mean().sort_values(ascending=False)


LotFrontage     0.177397
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
                  ...   
BsmtUnfSF       0.000000
TotalBsmtSF     0.000000
MSZoning        0.000000
HeatingQC       0.000000
SalePrice       0.000000
Length: 75, dtype: float64

# 3 For numerical columns, fill in any missing data with the median value.

In [8]:
housing_df.isnull().sum().sort_values(ascending=False)

LotFrontage     259
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
               ... 
BsmtUnfSF         0
TotalBsmtSF       0
MSZoning          0
HeatingQC         0
SalePrice         0
Length: 75, dtype: int64

In [9]:
#Filling in missing feilds with median for numerical columns
housing_df['LotFrontage'].fillna(value=housing_df['LotFrontage'].median(), inplace=True)
housing_df['GarageYrBlt'].fillna(value=housing_df['GarageYrBlt'].median(), inplace=True)
housing_df['MasVnrArea'].fillna(value=housing_df['MasVnrArea'].median(), inplace=True)


# 4. For categorical columns, fill in any missing data with the most common value (mode).

In [10]:
#Filling in missing feilds with mode for categorical columns
#mode()[0] find the most occuring value
housing_df['GarageType'].fillna(value=housing_df['GarageType'].mode()[0], inplace=True)
housing_df['GarageCond'].fillna(value=housing_df['GarageCond'].mode()[0], inplace=True)
housing_df['GarageFinish'].fillna(value=housing_df['GarageFinish'].mode()[0], inplace=True)
housing_df['GarageQual'].fillna(value=housing_df['GarageQual'].mode()[0], inplace=True)
housing_df['BsmtFinType2'].fillna(value=housing_df['BsmtFinType2'].mode()[0], inplace=True)
housing_df['BsmtFinType1'].fillna(value=housing_df['BsmtFinType1'].mode()[0], inplace=True)
housing_df['BsmtExposure'].fillna(value=housing_df['BsmtExposure'].mode()[0], inplace=True)
housing_df['BsmtQual'].fillna(value=housing_df['BsmtQual'].mode().mode()[0], inplace=True)
housing_df['BsmtCond'].fillna(value=housing_df['BsmtCond'].mode().mode()[0], inplace=True)
housing_df['Electrical'].fillna(value=housing_df['Electrical'].mode().mode()[0], inplace=True)
housing_df['MasVnrType'].fillna(value=housing_df['MasVnrType'].mode().mode()[0], inplace=True)


In [11]:
#No more null values
housing_df.isnull().sum().sort_values(ascending=False)

MSSubClass      0
GarageYrBlt     0
Fireplaces      0
Functional      0
TotRmsAbvGrd    0
               ..
MasVnrArea      0
MasVnrType      0
Exterior2nd     0
Exterior1st     0
SalePrice       0
Length: 75, dtype: int64

# 5.Convert the categorical columns to dummy variables

In [12]:
#Using pd get dummy to convert to dummy values
housing_df_dummmy = pd.get_dummies(housing_df)

# 6. Split the data into a training and test set, where the SalePrice column is the target.

In [13]:
#Splitting sales as target
target = housing_df_dummmy['SalePrice']
#Splitting features and dropping salesprice
features = housing_df_dummmy.drop('SalePrice', axis=1).copy()

In [14]:
#Training the data 
#set the test_size to .20
features_train, features_test, target_train, target_test = train_test_split(features, target,  shuffle=True, test_size = 0.20)

# 7. Run a linear regression and report the R2-value and RMSE on the test set.

In [15]:
#Making a function to return r2 
def RsquaredConversion(target_test, target_predicated):
    #Invokes r2 score function 
    r_squared = r2_score(target_test,target_predicated)
    #Returns value
    return r_squared


In [16]:
#Instantiating linear regression class
regression = LinearRegression()

In [17]:
#Fitting model with training data 
model = regression.fit(features_train,target_train)

In [19]:
target_predicted = model.predict(features_test)

In [20]:
#Get R2 score
r2 = RsquaredConversion(target_test,target_predicted)
print(r2)

0.5220918955280652


In [22]:
#Getting mse
mse = mean_squared_error(target_test, target_predicted)
print(mse)

3233901426.780022


In [23]:
#Getting rmse
rmse = np.sqrt(mse)
print(rmse)

56867.40214551762


# 8. Fit and transform the training features with a PCA so that 90% of the variance is retained (see section 9.1 in the Machine Learning with Python Cookbook).

In [24]:
#90% variance is retained
pca = PCA(n_components=0.90, whiten=True)

In [25]:
train_features_pca = pca.fit_transform(features_train)

# 9.How many features are in the PCA-transformed matrix?

In [26]:
print("Original number of features:", features.shape[1])
print("Reduced number of features:", train_features_pca.shape[1])

Original number of features: 270
Reduced number of features: 1


# 10. Transform but DO NOT fit the test features with the same PCA.

In [27]:
#transforming pca
test_features_pca = pca.transform(features_test)

In [28]:
print("Reduced number of features:", test_features_pca.shape[1])

Reduced number of features: 1


# 11. Repeat step 7 with your PCA transformed data.

In [44]:
#repeating model steps

In [29]:
pca_model = regression.fit(train_features_pca,target_train)

In [30]:
target_predicted = model.predict(test_features_pca)

In [31]:
r2 = RsquaredConversion(target_test,target_predicted)
print(r2)

-0.08792648987385876


In [32]:
mse = mean_squared_error(target_test, target_predicted)
print(mse)

7361764730.318489


In [33]:
#Getting rmse
rmse = np.sqrt(mse)
print(rmse)

85800.72686357901


# 12.Take your original training features (from step 6) and apply a min-max scaler to them.

In [34]:
MinMaxScaler(copy=True, feature_range=(1))

#Setting min max scaler
scaler = MinMaxScaler()

#features_train
features_train_std = scaler.fit_transform(features_train)


# 13. Find the min-max scaled features in your training set that have a variance above 0.1 (see Section 10.1 in the Machine Learning with Python Cookbook).

In [35]:
#Setting threshold for .1 
thresholder = VarianceThreshold(threshold=.1)

In [36]:
#fit and transform training data
feature_train_scaled = thresholder.fit_transform(features_train_std)

# 14.Transform but DO NOT fit the test features with the same steps applied in steps 11 and 12.

In [37]:
#test_features_pca = pca.transform(features_test)
features_test_std = scaler.transform(features_test)

In [38]:
#scaling data with threshold
features_test_scaled = thresholder.transform(features_test_std )

# 15.Repeat step 7 with the high variance data.

In [39]:
#Fitting model with training data 
#Fitting with train scaled 
model = regression.fit(feature_train_scaled ,target_train)


In [40]:
#predicting with features scaled 
target_predicted = model.predict(features_test_scaled)

In [46]:
#repeating model steps

In [41]:
r2 = RsquaredConversion(target_test,target_predicted)
print(r2)

0.6319484318636891


In [42]:
mse = mean_squared_error(target_test, target_predicted)
print(mse)

2490525856.722603


In [43]:
rmse = np.sqrt(mse)
print(rmse)

49905.16863735261


# 16.Summarize your findings.

With the untampered training and testing sets the R2 score was 52 and the rmse is high. When I used the PCA model, the r2 score with -0.087 the rmse was higher than the untampered set. The min max model had a R2 score of 63 with a lower rmse then both.  The first model indicates that the relationship between these variables explain 52 percent of the variation in the data. With the Pca model the r2 indicates that the given data does not follow the trend of the data. The last model of 63 is similar to the untampered data. The rmse indicates a high level of over fitting. This can be due to the dummy variable classification that  gives us more columns. The one with the biggest difference was PCA.